Preprocessing all comment files in the directory: removing NaNs and duplicates, deleted comments, normalizing author names, converting all texts to lower case, and merging them into one csv file with pre-processed submissions.

This can be quite computationally expensive, so we advice using cloud services for the task.

In [ ]:
import pandas as pd
import re
import glob
import os

In [ ]:
comments_dir = "/PATH/comments"
submissions = "/PATH/merged_submissions.csv"

output_path = os.path.join("/OUTPUT_PATH", "merged_dataset.csv")

files = glob.glob(os.path.join(comments_dir, "*.csv"))

cleaned_dfs = []

In [ ]:
def clean_comments(df):
    df = df.dropna(subset=["body"]).copy()
    df = df.drop_duplicates(subset=["author", "body", "created"])

    df["body"] = (
        df["body"]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(r"[^a-zA-Z0-9\s'.,!?-]", "", regex=True)
    )

    df = df[~df["body"].isin(["removed", "deleted"])]

    df["author"] = df["author"].astype(str).str.strip().str.replace(r"^u/", "", regex=True)

    return df

In [ ]:
files

In [ ]:
for file in files:
    print(f"Cleaning {file} ...")
    df = pd.read_csv(file)
    cleaned = clean_comments(df)
    cleaned_dfs.append(cleaned)

In [ ]:
merged_comments = pd.concat(cleaned_dfs, ignore_index=True)

In [ ]:
submissions_df = pd.read_csv(submissions)
submissions_df = submissions_df.rename(columns={"text": "body"}) # normalizing columns for consistency

In [ ]:
merged_final = pd.concat([merged_comments, submissions_df], ignore_index=True)
merged_final = merged_final.drop_duplicates(subset=["author", "body", "created"])

In [ ]:
merged_final["created"] = pd.to_datetime(merged_final["created"], errors="coerce")
merged_final = merged_final.sort_values(by="created", ascending=True).reset_index(drop=True)

In [ ]:
len(merged_final)

In [ ]:
merged_final.dtypes.value_counts()

In [ ]:
num_posts = merged_final["title"].notna().sum()
num_comments = merged_final["title"].isna().sum()

print(f"Comments: {num_comments:,}")
print(f"Submissions: {num_posts:,}")

In [ ]:
merged_final.info()

In [ ]:
merged_final.head()

In [ ]:
merged_final.to_csv(output_path, index=False)